# Identify gaps in CARE service relative to unsheltered population

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
catalog = intake.open_catalog('../catalogs/*.yml')

## Merge homelessness and population

In [3]:
df = gpd.read_file('s3://city-of-los-angeles-data-lake/public-health-dashboard/gis/raw/homelessness_lacity_2017_2019.geojson')
pop = pd.read_parquet('s3://city-of-los-angeles-data-lake/public-health-dashboard/data/raw/pop_by_tract2017.parquet') 

In [4]:
df.head()

,index,Tract,SPA,SD,CD,totUnshelt,totShelt,totPeople,year,GEOID,full_area,clipped_area,geometry
0,0,121210,2,3,6,43.585,0,43.585,2018,06037121210,0.778304,0.778304,"POLYGON ((6442725.264 1909753.110, 6442659.151..."
1,0,121210,2,3,6,43.585,0,43.585,2018,06037121210,0.778304,0.778304,"POLYGON ((6442725.264 1909753.110, 6442659.151..."
2,0,900505,1,5,0,17.275,0,17.275,2019,06037900505,0.505467,NaN,None
3,0,900505,1,5,0,17.275,0,17.275,2019,06037900505,0.505467,NaN,None
4,0,980024,2,3,6,253.647,0,253.647,2017,06037980024,3.199074,3.199074,"POLYGON ((6418636.803 1889623.260, 6419650.324..."


In [5]:
pop.head()

,GEOID,pop
1074,06037101110,4566.0
291,06037101122,3064.0
2232,06037101210,6043.0
2094,06037101220,3340.0
1060,06037101300,4285.0


In [7]:
m1 = pd.merge(df, pop, on = 'GEOID', how = 'left', validate = 'm:1')

In [8]:
m1.head()

,index,Tract,SPA,SD,CD,totUnshelt,totShelt,totPeople,year,GEOID,full_area,clipped_area,geometry,pop
0,0,121210,2,3,6,43.585,0,43.585,2018,06037121210,0.778304,0.778304,"POLYGON ((6442725.264 1909753.110, 6442659.151...",3095.0
1,0,121210,2,3,6,43.585,0,43.585,2018,06037121210,0.778304,0.778304,"POLYGON ((6442725.264 1909753.110, 6442659.151...",3095.0
2,0,900505,1,5,0,17.275,0,17.275,2019,06037900505,0.505467,NaN,None,3732.0
3,0,900505,1,5,0,17.275,0,17.275,2019,06037900505,0.505467,NaN,None,3732.0
4,0,980024,2,3,6,253.647,0,253.647,2017,06037980024,3.199074,3.199074,"POLYGON ((6418636.803 1889623.260, 6419650.324...",258.0


## Map homelessness

In [ ]:

fig, ax = plt.subplots(figsize = (15, 10))
gdf2.plot(column='totUnshelt', ax=ax, legend=True)